In [32]:
#text_file = "eng-ban.txt"
text_file = "ban-eng.txt"
# To open the text file we need to encode the text. Here, we use 'utf8' encoding
with open(text_file, encoding="utf8") as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    bangla, english = line.split(" >>> ")
    bangla = "[start] " + bangla + " [end]"
    text_pairs.append((english, bangla))

In [33]:
import random
for i in range(len(text_pairs)):
    print(random.choice(text_pairs));

('About seventy percentage of people are literate.', '[start] বাংলাদেশের প্রায় সত্তর ভাগ মানুষ স্বাক্ষর. [end]')
('Bangla is a language.', '[start] বাংলা একটি ভাষা. [end]')
('21st of February is the International Mother Language Day.', '[start] একুশে ফেব্রুয়ারি আন্তর্জাতিক মাতৃভাষা দিবস. [end]')
('I know Bangla.', '[start] আমি বাংলা জানি. [end]')
('Sorry.', '[start] দুঃখিত. [end]')
('About hundrad million people speaks in Bangla.', '[start] প্রায় এক কোটি মানুষ বাংলায় কথা বলে. [end]')
('Sorry.', '[start] দুঃখিত. [end]')
('No.', '[start] না. [end]')
('About hundrad million people speaks in Bangla.', '[start] প্রায় এক কোটি মানুষ বাংলায় কথা বলে. [end]')
('I know Bangla.', '[start] আমি বাংলা জানি. [end]')
('Sorry.', '[start] দুঃখিত. [end]')
('It is a Bangla to English translation file.', '[start] এটা বাংলা থেকে ইংরেজি অনুবাদের নথি. [end]')
('About hundrad million people speaks in Bangla.', '[start] প্রায় এক কোটি মানুষ বাংলায় কথা বলে. [end]')
('Bangla is a language.', '[start] বাংলা একটি ভাষ

In [34]:
import random
random.shuffle(text_pairs)
total_data_length = len(text_pairs)
num_val_samples = int(0.15 * total_data_length)

In [35]:
num_train_samples = (total_data_length - (2 * num_val_samples))
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples]

In [36]:
import tensorflow as tf
import string
import re

strip_chars = string.punctuation + "?"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

In [7]:
f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~\\?]'

In [37]:
# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
from tensorflow.keras import layers

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return(tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", ""
    ))

vocab_size = 10000
sequence_length = 20
source_vectorization = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length
)
target_vectorization = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length + 1,
    standardize = custom_standardization
)

In [38]:
train_english_texts = [pair[0] for pair in train_pairs]
train_bangla_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_bangla_texts)

In [40]:
batch_size = 64

def format_dataset(eng, ban):
    eng = source_vectorization(eng)
    ban = source_vectorization(ban)
    return({"english": eng, "bangla": ban[:, :-1]}, ban[:, 1:])

def make_dataset(pairs):
    eng_texts, ban_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    ban_texts = list(ban_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, ban_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls = 4)
    return(dataset.shuffle(2048).prefetch(16).cache())

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [22]:
#print(list(train_ds.as_numpy_iterator()));

In [41]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['bangla'].shape: {inputs['bangla'].shape}")
    print(f"target.shape: {targets.shape}")

inputs['english'].shape: (12, 20)
inputs['bangla'].shape: (12, 19)
target.shape: (12, 19)


In [42]:
from tensorflow import keras

embed_dim = 256
latent_dim = 1024

In [43]:
source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(layers.GRU(latent_dim), merge_mode="sum")(x)

In [44]:
past_target = keras.Input(shape=(None,), dtype="int64", name="bangla")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

In [29]:
#seq2seq_rnn.summary()

In [49]:
seq2seq_rnn.compile(
    optimizer = "rmsprop",
    loss = "sparse_categorical_crossentropy",
    #metrices = ["accuracy"]
)
seq2seq_rnn.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/5
1/1 [==============================] - 21s 21s/step - loss: 2.9485 - val_loss: 3.0188
Epoch 2/5
1/1 [==============================] - 2s 2s/step - loss: 2.8170 - val_loss: 1.6432
Epoch 3/5
1/1 [==============================] - 2s 2s/step - loss: 1.3002 - val_loss: 0.2398
Epoch 4/5
1/1 [==============================] - 2s 2s/step - loss: 0.3260 - val_loss: 0.4579
Epoch 5/5
1/1 [==============================] - 2s 2s/step - loss: 0.5348 - val_loss: 0.2541
